In [ ]:
###
###テキスト分析1_セル1
###

#シェイクスピアの作品分類
#テキストがGoogle Collaborator のファイルの保存位置に入る
import os
folder='texts'
if not os.path.isdir(folder):
  os.mkdir(folder)
!wget -O $folder/shakespeares-works_TXT_FolgerShakespeare.zip https://flgr.sh/txtfssAlltxt?_ga=2.104281097.80727893.1702953543-617012875.1702953542
!unzip -d $folder -o texts/shakespeares-works_TXT_FolgerShakespeare.zip
#終わりの何行を比較対象とするか
amount_of_last_lines=50



In [ ]:
###
###テキスト分析1_セル2
###

#テクストを読み取り、前処理
import glob
import re
#前処理済みのテクストを格納する変数
titles=[]
plays=[]
#詩作品は除外
exclude_poems=['the-phoenix-and-turtle','venus-and-adonis','shakespeares-sonnets','lucrece']
#ディレクトリの中のテクストを一つずつ読み込む　例外：shakespeares-sonnets
for v in glob.glob(folder+'/*.txt'):
  #詩は除外
  title=v[len(folder)+1:v.find('_')]
  if title in exclude_poems:
    continue
  with open(v) as f:
    text=f.readlines()
  print(v)
#前処理1: 前書き（テクストの説明から登場人物までINDUCTION含む）とあとがきEPILOGUEを消す
  key=['ACT 1\n']
  text2=[]
  flag=0
  for i in text:
    if i in key:
      flag=1
    #ACT 1 より前までとEPILOGUEより後が消える
    if re.search(r'^EPILOGUE.*\n$', i)!=None:
      flag=0
    if flag!=0:
      text2.append(i)
  text3=[]
  togaki_flag=False
#前処理2: ACT1 Scene1(=, Chorus含む)といった文字を消す
  for i in text2:
    if re.search(r'^ACT [0-9].*\n$', i)!=None:
      i=''
    if re.search(r'^Scene [0-9].*\n$', i)!=None:
      i=''
    if re.search(r'^==.*\n$', i)!=None:
      i=''
    if re.search(r'^[0-9].*Chorus\n$', i)!=None:
      i=''
#前処理3: ト書き[]を消す
    #ト書き[]が1行内にある場合
    m=re.findall('\[[^\]]+\]',i)
    for j in m:
      i=i.replace(j,'')
    #ト書き[]が複数行にわたってある場合
    m=i.find('[')
    if m!=-1:
      i=re.sub('\[.*','',i)
      togaki_flag=True
    if togaki_flag:
      m=i.find(']')
      #その行全体がまだト書きの場合
      if m==-1:
        i=''
      #その行に終わりの]がある場合
      if m!=-1:
        i=i[m+1:]
        togaki_flag=False
#前処理4: 人物名を消す THESEUS  など。※ I に注意
    #人物の名前で始まる行
    m=re.search(r'^[A-Z]* *', i)
    if m!=None and len(m.group())>2:
      i=re.sub('^[A-Z]* *','', i)
    #人物の名前だけで改行している場合
    m=re.search(r'^[A-Z]*$', i)
    if m!=None and len(m.group())>1:
      i=re.sub('^[A-Z]*$','', i)
#前処理5: その他不要文字
    i=re.sub('\t','', i)
#前処理6: 改行を消す
    if i!='\n' and i!='':
      text3.append(i)
  titles.append(title)
  plays.append(text3)

#確認
print(titles)
print(plays[0])